In [1]:
from google.colab import drive
import zipfile
import os


drive.mount('/content/drive')

zip_path = '/content/drive/MyDrive/apredizajeautomatico/QuickDraw-Animals.zip'  # Cambia esta ruta
zip_path_2 = '/content/drive/MyDrive/apredizajeautomatico/QuickDraw-10-Tarea2.zip'

extract_path = '/content/imagenes_descomprimidas'
os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

with zipfile.ZipFile(zip_path_2, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Archivos extraídos en: {extract_path}")

print("Archivos extraídos:")
print(os.listdir(extract_path))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Archivos extraídos en: /content/imagenes_descomprimidas
Archivos extraídos:
['QuickDraw-Animals', 'QuickDraw-10']


In [4]:
# importacion de librerias
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from tensorflow.keras.utils import to_categorical


# Parte 1 - Lectura de Imágenes

In [3]:
def load_image_paths_and_labels(file_path):
    """Carga las rutas de las imágenes y las etiquetas desde un archivo de texto."""
    image_paths = []
    labels = []
    with open(file_path, 'r') as file:
        for line in file.readlines():
            path, label = line.strip().split('\t')
            image_paths.append(path)
            labels.append(int(label))
    return image_paths, np.array(labels)

def load_images(image_paths, folder_route):
    """Carga las imágenes y las aplana a vectores."""
    images = []
    for path in image_paths:
        with Image.open(folder_route + path) as img:
            img_array = np.array(img).reshape(-1)
            images.append(img_array)
    return np.array(images)

QuickDraw-10

In [4]:
# Cargar las rutas de entrenamiento y prueba junto con las etiquetas
train_image_paths_10, train_labels_10 = load_image_paths_and_labels("/content/imagenes_descomprimidas/QuickDraw-10/train.txt")
test_image_paths_10, test_labels_10 = load_image_paths_and_labels("/content/imagenes_descomprimidas/QuickDraw-10/test.txt")

# Cargar y procesar las imágenes
train_images_10 = load_images(train_image_paths_10, "/content/imagenes_descomprimidas/QuickDraw-10/")
test_images_10 = load_images(test_image_paths_10, "/content/imagenes_descomprimidas/QuickDraw-10/")

In [5]:
# Preparacion datos de validacion
train_images_10, val_image_10, train_labels_10, val_labels_10 = train_test_split(
    train_images_10, train_labels_10, test_size=0.15, random_state=42, stratify=train_labels_10
)

In [6]:
# normalizacion
train_images_10 = train_images_10.astype('float32') / 255.0
test_images_10 = test_images_10.astype('float32') / 255.0
val_image_10 = val_image_10.astype('float32') / 255.0

In [7]:
# one hot encoding
n_classes=10
train_labels_10 = to_categorical(train_labels_10, num_classes=n_classes)
test_labels_10 = to_categorical(test_labels_10, num_classes=n_classes)
val_labels_10 = to_categorical(val_labels_10, num_classes=n_classes)

QuickDraw-Animals

In [8]:
def load_images_and_labels(dataset):
    """
    Carga imágenes y etiquetas para train y test del dataset QuickDraw-Animals.

    Retorna:
      X_test, y_test, X_train, y_train (numpy arrays)
    """
    if dataset != "Animals":
        raise ValueError("Sólo soporta dataset 'Animals'.")

    base_path = '/content/imagenes_descomprimidas/QuickDraw-Animals/'
    mapping_file = os.path.join(base_path, 'mapping.txt')

    # Leer mapping.txt y crear diccionario etiqueta->número
    label_map = {}
    with open(mapping_file, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) == 2:
                label = parts[0]
                idx = int(parts[1])
                label_map[label] = idx

    def load_images_from_folder(folder_path):
        images = []
        labels = []
        # Las subcarpetas son las clases
        for label_name in sorted(os.listdir(folder_path)):
            label_folder = os.path.join(folder_path, label_name)
            if os.path.isdir(label_folder) and label_name in label_map:
                for img_file in sorted(os.listdir(label_folder)):
                    if img_file.lower().endswith(('.png', '.jpg', '.jpeg')):
                        img_path = os.path.join(label_folder, img_file)
                        with Image.open(img_path) as img:
                            img_array = np.array(img).reshape(-1)
                            images.append(img_array)
                            labels.append(label_map[label_name])
        return np.array(images), np.array(labels)

    # Cargar test
    X_test, y_test = load_images_from_folder('/content/imagenes_descomprimidas/QuickDraw-Animals/test_images/test_images/')

    # Cargar train
    X_train, y_train = load_images_from_folder('/content/imagenes_descomprimidas/QuickDraw-Animals/train_images/train_images/')

    return X_test, y_test, X_train, y_train


In [9]:
test_images_animals, test_labels_animals, train_images_animals, train_labels_animals = load_images_and_labels("Animals")

In [10]:
# Preparacion datos de validacion
train_images_animals, val_images_animals, train_labels_animals, val_labels_animals = train_test_split(
    train_images_animals, train_labels_animals, test_size=0.15, random_state=42, stratify=train_labels_animals
)

In [ ]:
# normalizacion
train_images_animals = train_images_animals.astype('float32') / 255.0
test_images_animals = test_images_animals.astype('float32') / 255.0
val_images_animals = val_images_animals.astype('float32') / 255.0

In [ ]:
# one hot encoding
n_classes=12
train_labels_animals = to_categorical(train_labels_animals, num_classes=n_classes)
test_labels_animals = to_categorical(test_labels_animals, num_classes=n_classes)
val_labels_animals = to_categorical(val_labels_animals, num_classes=n_classes)

## Parte 2 - Construcción de modelos

Clase MLP

In [7]:
class MLP(tf.keras.Model):
    # defining components
    def __init__(self, layers_size, n_classes, activation='sigmoid'):
        super(MLP, self).__init__()
        self.layer_list = []
        for lsize in layers_size:
            self.layer_list.append(tf.keras.layers.Dense(lsize))
        self.classifier = tf.keras.layers.Dense(n_classes)
        self.activation = activation


    # defining architecture
    def call(self, inputs):
        x = inputs
        for mlp_layer in self.layer_list:
            x = mlp_layer(x)
            if self.activation == 'sigmoid':
                x = tf.keras.activations.sigmoid(x)
            elif self.activation == 'relu':
                x = tf.keras.activations.relu(x)
            elif self.activation == 'tanh':
                x = tf.keras.activations.tanh(x)
            elif self.activation == 'leakyrelu':
                x = tf.keras.activations.leakyrelu(x)
            else:
                raise ValueError("Activación no soportada")
        x = self.classifier(x)
        return tf.keras.activations.softmax(x)

Funciones para entrenar, evaluar y realizar los experimentos

In [8]:
def train_model(model, X_train, y_train, X_val, y_val, loss_fn, epochs=10, batch_size=32):

    # Entrenamiento
    model.compile(
        optimizer=tf.keras.optimizers.SGD(),
        loss=loss_fn,
        metrics=['accuracy']
    )

    # Detiene el proceso tras 5 epocas sin encontrar una mejora. Evalúa la pérdida en el
    # conjunto de validación.
    early_stopping = EarlyStopping(
      monitor='val_loss',
      patience=5,
      restore_best_weights=True,
      verbose=1
    )

    final_model = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        # verbose=0
    )
    return final_model

def evaluate_model(model, X_test, y_test, class_names):
    y_pred_probs = model(X_test, training=False).numpy()
    y_pred = np.argmax(y_pred_probs, axis=1)

    # Si y_test es one-hot, convierte a etiquetas enteras
    if len(y_test.shape) > 1 and y_test.shape[1] > 1:
        y_test_int = np.argmax(y_test, axis=1)
    else:
        y_test_int = y_test

    acc_total = accuracy_score(y_test_int, y_pred)

    acc_per_class = {}
    for cls in np.unique(y_test_int):
        idx = y_test_int == cls
        acc = accuracy_score(y_test_int[idx], y_pred[idx])
        acc_per_class[class_names[cls]] = acc

    cm = confusion_matrix(y_test_int, y_pred)

    return acc_total, acc_per_class, cm


def experiment(X_train, y_train, X_val, y_val, X_test, y_test, layers_size, n_classes, activation, loss_fn, class_names, epochs=10, batch_size=None, n_experiments=5):
    acc_totals = []
    acc_classes_list = []
    cm_list = []

    for i in range(n_experiments):
        print(f"\nEntrenamiento número {i+1}")
        # Crear modelo nuevo para reinicializar pesos
        model = MLP(layers_size, n_classes, activation)
        model.build(input_shape=(None, X_train.shape[1]))

        # Entrenar
        train_model(model, X_train, y_train, X_val, y_val, loss_fn, epochs, batch_size)

        # Evaluar
        acc_total, acc_per_class, cm = evaluate_model(model, X_test, y_test, class_names)
        acc_totals.append(acc_total)
        acc_classes_list.append(acc_per_class)
        cm_list.append(cm)

    # Calcular mediana del accuracy total
    median_acc_total = np.median(acc_totals)
    print(f"\nMediana de accuracy total tras 5 entrenamientos: {median_acc_total:.4f}")

    return median_acc_total, acc_classes_list, cm_list

Experimentos para QuickDraw-10

In [14]:
clases = ['bandage', 'blackberry', 'castle', 'flashlight', 'lion', 'remote-control', 'sink', 'spreadsheet', 'teapot', 'trombone']

In [ ]:
# Modelo 1 - QuickDraw-10
# Detalles:
# Activacion: sigmoid
# Perdida: CrossEntropy
# Capas: 3
# Epocas: 100
# Batch size: 100

acc_total, acc_clase, cm = experiment(
        train_images_10,
        train_labels_10,
        val_image_10,
        val_labels_10,
        test_images_10,
        test_labels_10,
        layers_size=[512, 256,128],
        n_classes=10,
        activation='sigmoid',
        loss_fn='categorical_crossentropy',
        class_names=clases,
        epochs=100,
        batch_size=100,
        n_experiments=5
        )

In [ ]:
acc_clase
acc_total
cm[0]

In [1]:
# Modelo 2 - QuickDraw-10
# Detalles:
# Activacion: sigmoid
# Perdida: CrossEntropy
# Capas: 3
# Epocas: 20
# Batch size: 1000

acc_total, acc_clase, cm = experiment(
        train_images_10,
        train_labels_10,
        val_image_10,
        val_labels_10,
        test_images_10,
        test_labels_10,
        layers_size=[512, 256, 128],
        n_classes=10,
        activation='sigmoid',
        loss_fn='categorical_crossentropy',
        class_names=clases,
        epochs=20,
        batch_size=1000,
        n_experiments=5
        )

NameError: name 'experiment' is not defined

In [ ]:
# Modelo 3 - QuickDraw-10
# Detalles:
# Activacion: relu
# Perdida: Categorical Hinge
# Capas: 4
# Epocas: 60
# Batch size: 550

acc_total, acc_clase, cm = experiment(
        train_images_10,
        train_labels_10,
        val_image_10,
        val_labels_10,
        test_images_10,
        test_labels_10,
        layers_size=[512, 256, 128, 64],
        n_classes=10,
        activation='relu',
        loss_fn='categorical_hinge',
        class_names=clases,
        epochs=60,
        batch_size=550,
        n_experiments=5
        )

Experimentos para QuickDraw-Animals

In [9]:
clases = ['sheep', 'bear', 'bee', 'cat', 'camel', 'cow', 'crab', 'crocodile', 'duck', 'elephant', 'dog', 'giraffe']

In [ ]:
# Modelo 1 - QuickDraw-Animals
# Detalles:
# Activacion: tanh
# Perdida: categorical_hinge
# Capas: 2
# Epocas: 60
# Batch size: 550

acc_total, acc_clase, cm = experiment(
        train_images_animals,
        train_labels_animals,
        val_images_animals,
        val_labels_animals,
        test_images_animals,
        test_labels_animals,
        layers_size=[512, 256],
        n_classes=12,
        activation='tanh',
        loss_fn='categorical_hinge',
        class_names=clases,
        epochs=10,
        batch_size=1000,
        n_experiments=5
        )

In [ ]:
# Modelo 2 - QuickDraw-Animals
# Detalles:
# Activacion: tanh
# Perdida: categorical_crossentropy
# Capas: 2
# Epocas: 10
# Batch size: 1000

acc_total, acc_clase, cm = experiment(
        train_images_animals,
        train_labels_animals,
        val_images_animals,
        val_labels_animals,
        test_images_animals,
        test_labels_animals,
        layers_size=[512, 256, 128],
        n_classes=12,
        activation='tanh',
        loss_fn='categorical_crossentropy',
        class_names=clases,
        epochs=100,
        batch_size=150,
        n_experiments=5
        )

In [ ]:
# Modelo 3 - QuickDraw-Animals
# Detalles:
# Activacion: sigmoid
# Perdida: kullback_leibler_divergence
# Capas: 2
# Epocas: 10
# Batch size: 1000

acc_total, acc_clase, cm = experiment(
        train_images_animals,
        train_labels_animals,
        val_images_animals,
        val_labels_animals,
        test_images_animals,
        test_labels_animals,
        layers_size=[512, 256, 128, 64],
        n_classes=12,
        activation='sigmoid',
        loss_fn='kullback_leibler_divergence',
        class_names=clases,
        epochs=10,
        batch_size=20,
        n_experiments=5
        )